In [1]:
import numpy as np
import pandas as pd 

In [2]:
df=pd.read_csv('cleaned_data.csv')

In [3]:
df

,location,total_sqft,bath,price,BHK
0,Electronic City Phase II,1056.0,2.0,39.07,2
1,Chikka Tirupathi,2600.0,5.0,120.00,4
2,Uttarahalli,1440.0,2.0,62.00,3
3,Lingadheeranahalli,1521.0,3.0,95.00,3
4,Kothanur,1200.0,2.0,51.00,2
...,...,...,...,...,...
11557,Bellandur,1262.0,2.0,47.00,2
11558,Green Glen Layout,1715.0,3.0,112.00,3
11559,Whitefield,3453.0,4.0,231.00,5
11560,other,3600.0,5.0,400.00,4


In [4]:
df.dtypes

location       object
total_sqft    float64
bath          float64
price         float64
BHK             int64
dtype: object

In [5]:
df.isnull().sum()


location      0
total_sqft    0
bath          0
price         0
BHK           0
dtype: int64

In [6]:
location_counts = df['location'].value_counts()

rare_locations = location_counts[location_counts <= 10].index

df['location'] = df['location'].apply(
    lambda x: 'other' if x in rare_locations else x
)


In [7]:
print(df.columns)


Index(['location', 'total_sqft', 'bath', 'price', 'BHK'], dtype='object')


In [8]:
df = pd.get_dummies(df, columns=['location'], drop_first=True)


In [9]:
x=df.drop(['price'],axis=1)
y=df['price']

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error


In [11]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [12]:
rf=RandomForestRegressor( n_estimators=200,
    random_state=42,
    n_jobs=-1)
rf.fit(x_train,y_train)
y_pred=rf.predict(x_test)
print("RF MAE:", mean_absolute_error(y_test, y_pred))



RF MAE: 23.838680737932116


In [13]:
extreme = pd.DataFrame({
    'location': ['other'],
    'total_sqft': [500],
    'bath': [2],
    'BHK': [5]
})
extreme = pd.get_dummies(extreme, columns=['location'], drop_first=True)
extreme = extreme.reindex(columns=x.columns, fill_value=0)


pred_lakhs = rf.predict(extreme)[0]

def format_price(lakhs):
    if lakhs >= 100:
        return f"₹{lakhs/100:.2f} Crore"
    else:
        return f"₹{lakhs:.0f} Lakh"

print("Predicted Price:", format_price(pred_lakhs))


Predicted Price: ₹99 Lakh
